# *Лабораторная работа №3*

## Построение регрессора для предсказания непрерывной величины (Теоретическая часть)

В данной лабораторной работе будет рассматриваться построение регрессионной модели для предсказания непрерывной величины. До текущего момента мы сталкивались только с задачей определения классовой принадлежности объекта, то есть в качестве результата должно было быть получено число из конечного множества целых чисел.

Подключим все необходимые библиотеки.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense

import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Для примера построения регрессионной модели возьмем датасет, работу по очистке которого мы производили в рамках первой лабораторной работы (база данных по продажам подержанных автомобилей в Германии).

In [ ]:
%%capture
!wget https://www.dropbox.com/s/s1sqfsi6x7hbs28/autos_mod.csv

In [ ]:
df = pd.read_csv('autos_mod.csv', encoding='iso-8859-1')

In [ ]:
df.sample(10)

Удалим лишний столбец, возникший в результате записи датасета в файл. Кроме того, очистим базу от пустых значений, так как не все регрессионные методы могут обрабатывать поврежденные данные.

In [ ]:
del df['Unnamed: 0']
df_wo_na = df.dropna()
df_wo_na.reset_index(inplace=True)
del df_wo_na['index']

Проверим, что остались только необходимые нам столбцы.

In [ ]:
df_wo_na.sample(10)

Проведем факторизацию в столбцах со строковыми значениями. Для этого выпишем имена всех объектных столбцов и в цикле будем вызывать factorize из библиотеки pandas. [0] в конце строки с командой необходим так как factorize возвращает два набора значений - преобразованный столбец значений и упорядоченный список меток. Так как проводить операцию обратную факторизации мы не собираемся, то и второй столбец для нас не представляет интереса.

In [ ]:
column_names = ['vehicleType', 'gearbox', 'model', 'fuelType', 'brand', 'notRepairedDamage']
for i in column_names:
    df_wo_na.loc[:, i] = pd.factorize(df_wo_na.loc[:, i])[0]
    #df_wo_na[i] = pd.factorize(df_wo_na[i])[0]

In [ ]:
df_wo_na.head(3)

Разобьем выборку на тренировочный и тестовый наборы.

In [ ]:
Y = df_wo_na['price']
X = df_wo_na.drop(['price'], axis = 1)
train_points, test_points, train_values, test_values = train_test_split(X, Y, test_size = 0.2, random_state=7)

Обучим модель случайного леса из 100 решающих деревьев.

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, n_jobs=-1)
rf_model.fit(train_points, train_values)

Оценку эффективности нашей модели проведем через расчет MAE. MAE (Mean Absolute Error) - средняя абсолютная ошибка, рассчитанная как сумма абсолютных ошибок в каждом предсказании, деленная на общий размер выборки.

In [ ]:
rf_predict = rf_model.predict(test_points)
print(mean_absolute_error(test_values, rf_predict))

Разброс предсказнной цены в $5000+ не вызывает особого восторга. Попробуем произвести обучение других моделей и сравним результаты.

In [ ]:
nn_model = Sequential()
nn_model.add(Dense(9, activation='relu'))
nn_model.add(Dense(15, activation='relu'))
nn_model.add(Dense(15, activation='relu'))
nn_model.add(Dense(1, activation='linear'))

nn_model.compile(loss='mean_absolute_error', optimizer='adam')

results = nn_model.fit(
 train_points, train_values,
 epochs= 10,
 batch_size = 1000,
 validation_data = (test_points, test_values)
)

Значение функции потерь при обучении нейронной сети является MAE оценкой. Получили сопоставимый со случайным лесом результат. 

Напоследок, построим регрессор через градиентный бустинг.

In [ ]:
import xgboost as xgb
xg_reg = xgb.XGBRegressor(objective = 'reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 100, n_jobs=-1)

In [ ]:
xg_reg.fit(train_points, train_values)

In [ ]:
xgb_predict = xg_reg.predict(test_points)

Оценим качество получившейся модели.

In [ ]:
print(mean_absolute_error(test_values, xgb_predict))

Градиентный бустинг показал результаты еще хуже, чем были до этого... Попробуем проанализировать причины такого поведения.

Выведем график соответствия предсказанной и фактической цены.

In [ ]:
plt.figure(figsize=(7, 7))
plt.scatter(test_values, xgb_predict) # рисуем точки, соответствущие парам настоящее значение - прогноз
plt.plot([0, max(test_values)], [0, max(xgb_predict)], 'r') # рисуем прямую, на которой предсказания и настоящие значения совпадают
plt.xlabel('Настоящая цена', fontsize=20)
plt.ylabel('Предсказанная цена', fontsize=20);

Благодаря графическому представлению мы обнаружили сильнейшие вылеты максимальной цены автомобилей. Произведем анализ данного столбца.

In [ ]:
df_wo_na.hist(["price"])

На гистограмме видно, что у нас есть экземпляры, цена на которые доходит до 100 миллиардов. Произведем отбор записей по цене, исключив также записи со слишком низкой ценой.

In [ ]:
df_wo_na = df_wo_na[(df_wo_na['price'] < 20000) & (df_wo_na['price'] > 50)]
df_wo_na.reset_index(inplace=True)
del df_wo_na['index']

Выведем гисторгамму для распределения цен на автомобили после внесенных нами изменений.

In [ ]:
df_wo_na.hist(["price"])

Вновь проведем разделение на тренировочную и тестовую выборки.

In [ ]:
Y = df_wo_na['price']
X = df_wo_na.drop(['price'], axis = 1)
train_points, test_points, train_values, test_values = train_test_split(X, Y, test_size = 0.2)

Создадим модель градиентного бустинга на основе обновленных данных.

In [ ]:
xg_reg.fit(train_points, train_values)

In [ ]:
xgb_predict = xg_reg.predict(test_points)

In [ ]:
print(mean_absolute_error(test_values, xgb_predict))

Получившийся результат гораздо лучше с точки зрения отклонения абсолютной ошибки. Выведем график, чтобы оценить произошедшие изменения.

In [ ]:
plt.figure(figsize=(7, 7))
plt.scatter(test_values, xgb_predict) # рисуем точки, соответствущие парам настоящее значение - прогноз
plt.plot([0, max(test_values)], [0, max(xgb_predict)], 'r') # рисуем прямую, на которой предсказания и настоящие значения совпадают
plt.xlabel('Настоящая цена', fontsize=20)
plt.ylabel('Предсказанная цена', fontsize=20);

На графике явно видно, что наша модель показывает гораздо худшие резуьтаты для автомобилей с низкой стоимостью. Запомним этот факт и попробуем собрать модель на основе случайного леса, так как до проведения фильтрации по цене градиентный бустинг показывал худшие результаты.

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, n_jobs=-1)
rf_model.fit(train_points, train_values)
rf_predict = rf_model.predict(test_points)

In [ ]:
print(mean_absolute_error(test_values, rf_predict))

Средняя абсолютная ошибка меньше для модели случайного леса почти на $300, что является серьезным аргументом в ее пользу. Выведем график с отклонениями.

In [ ]:
plt.figure(figsize=(7, 7))
plt.scatter(test_values, rf_predict) # рисуем точки, соответствущие парам настоящее значение - прогноз
plt.plot([0, max(test_values)], [0, max(rf_predict)], 'r') # рисуем прямую, на которой предсказания и настоящие значения совпадают
plt.xlabel('Настоящая цена', fontsize=20)
plt.ylabel('Предсказанная цена', fontsize=20);

Теперь давайте разберемся в причинах того, что градиентный бустинг показывает результаты гораздо хуже случайного леса. Разгадка кроется в типах ансамблевых методов, которые лежат в основе данных моделей.

Случайный лес работает на основе бэггинга, то есть собирает ансамбль из слабых учеников, каждый из которых имеет собственный независимый набор данных и, как следствие, свой набор правил разделения записей. После получения решения от всех учеников принимается некое усредненное значение.

Градиентный бустинг же направлен на исправление результата самого слабого ученика. Это приводит к тому, что итоговая модель будет смещаться в направлении самых больших вылетов, что мы и видели на графиках выше.

Можно сделать вывод, что механизм градиентного бустинга наилучшим образом подходит для решения задач со взвешенным набором данных, т.е. с (примерно) равным количеством записей, относящихся к каждому классу, либо диапазону значений.

Проведем исследование также для нейросетевой модели.

In [ ]:
nn_model = Sequential()
nn_model.add(Dense(9, activation='relu'))
nn_model.add(Dense(15, activation='relu'))
nn_model.add(Dense(15, activation='relu'))
nn_model.add(Dense(1))

nn_model.compile(loss='mean_absolute_error', optimizer='adam')

results = nn_model.fit(
 train_points, train_values,
 epochs= 10,
 batch_size = 100,
 validation_data = (test_points, test_values)
)

nn_predict = nn_model.predict(test_points)
print(mean_absolute_error(test_values, nn_predict))

Теперь лидером нашего антирейтинга результатов стала модель, основанная на нейронных сетях.

Попробуем провести нормализацию данных и сранить результаты.

In [ ]:
df_norm = df_wo_na
Y = df_norm['price']
X = df_norm.drop(['price'], axis = 1)
for i in X.columns:
    X[i]=(X[i]-X[i].min())/(X[i].max()-X[i].min())

In [ ]:
X

In [ ]:
Y

In [ ]:
train_points, test_points, train_values, test_values = train_test_split(X, Y, test_size = 0.2)

In [ ]:
xg_reg.fit(train_points, train_values)

In [ ]:
xgb_predict = xg_reg.predict(test_points)

In [ ]:
print(mean_absolute_error(test_values, xgb_predict))

Положительная динамика в модели градиентного бустинга прослеживается, но она явно не претендует на роль ключевого фактора.

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, n_jobs=-1)
rf_model.fit(train_points, train_values)
rf_predict = rf_model.predict(test_points)

In [ ]:
print(mean_absolute_error(test_values, rf_predict))

А вот для случайного леса нормализация сыграла даже немного отрицательную роль. Связано это с тем, что при построении правил решающему дереву гораздо проще ориентироваться на натуральные числа, нежели на диапазон [0, 1].

In [ ]:
nn_model = Sequential()
nn_model.add(Dense(9, activation='relu'))
nn_model.add(Dense(15, activation='relu'))
nn_model.add(Dense(15, activation='relu'))
nn_model.add(Dense(1))

nn_model.compile(loss='mean_absolute_error', optimizer='adam')

results = nn_model.fit(
 train_points, train_values,
 epochs= 20,
 batch_size = 100,
 validation_data = (test_points, test_values)
)

nn_predict = nn_model.predict(test_points)
print(mean_absolute_error(test_values, nn_predict))

Для нейросетевой модели нормализация значительно облегчила задачу, но приблизиться к точности ансамблевых методов мы так и не смогли.

Рассмотрим популярность различных признаков при принятии решения в модели градиентного бустинга и попробуем удалить наиболее непопулярные.

In [ ]:
xgb.plot_importance(xg_reg)
plt.show()

In [ ]:
del df_wo_na['notRepairedDamage']
del df_wo_na['gearbox']
train_points, test_points, train_values, test_values = train_test_split(X, Y, test_size = 0.2)

Проверим влияние внесенных изменений на качество наших моделей.

In [ ]:
nn_model = Sequential()
nn_model.add(Dense(7, activation='relu'))
nn_model.add(Dense(15, activation='relu'))
nn_model.add(Dense(15, activation='relu'))
nn_model.add(Dense(1))

nn_model.compile(loss='mean_absolute_error', optimizer='adam')

results = nn_model.fit(
 train_points, train_values,
 epochs= 10,
 batch_size = 100,
 validation_data = (test_points, test_values)
)

nn_predict = nn_model.predict(test_points)
print(mean_absolute_error(test_values, nn_predict))

Точность нейросетевой модели, хоть и незначительно, но упала.

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, n_jobs=-1)
rf_model.fit(train_points, train_values)
rf_predict = rf_model.predict(test_points)

In [ ]:
print(mean_absolute_error(test_values, rf_predict))

In [ ]:
xg_reg.fit(train_points, train_values)
xgb_predict = xg_reg.predict(test_points)
print(mean_absolute_error(test_values, xgb_predict))

А вот ансамблевые методы показали небольшой рост точности.

Ну и напоследок проверим, каким образом скажется на качестве моделей разделение столбцов по способу факторизации.

In [ ]:
df_wo_na = df.dropna()
df_wo_na.reset_index(inplace=True)
del df_wo_na['index']

Для столбцов с высокой кардинальностью оставим стандартный алгоритм, а для столбцов с низкой применим OneHotEncoding.

In [ ]:
column_names_fact = ['vehicleType', 'model','brand']
column_names_dummies = ['gearbox', 'fuelType', 'notRepairedDamage']
for i in column_names_fact:
    df_wo_na[i] = pd.factorize(df_wo_na[i])[0]
df_wo_na = pd.get_dummies(df_wo_na, prefix=column_names_dummies)

In [ ]:
df_wo_na = df_wo_na[(df_wo_na['price'] < 20000) & (df_wo_na['price'] > 50)]
df_wo_na.reset_index(inplace=True)
del df_wo_na['index']

In [ ]:
Y = df_wo_na['price']
X = df_wo_na.drop(['price'], axis = 1)

In [ ]:
train_points, test_points, train_values, test_values = train_test_split(X, Y, test_size = 0.2)

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, n_jobs=-1)
rf_model.fit(train_points, train_values)
rf_predict = rf_model.predict(test_points)

In [ ]:
print(mean_absolute_error(test_values, rf_predict))

Модель случайного леса осталась равнодушна к увеличению количества столбцов.

In [ ]:
xg_reg.fit(train_points, train_values)
xgb_predict = xg_reg.predict(test_points)
print(mean_absolute_error(test_values, xgb_predict))

А вот градиентный бустинг значительно улучшил точность предсказываемой цены.

In [ ]:
nn_model = Sequential()
nn_model.add(Dense(17, activation='relu'))
nn_model.add(Dense(15, activation='relu'))
nn_model.add(Dense(15, activation='relu'))
nn_model.add(Dense(1))

nn_model.compile(loss='mean_absolute_error', optimizer='adam')

results = nn_model.fit(
 train_points, train_values,
 epochs= 10,
 batch_size = 100,
 validation_data = (test_points, test_values)
)

nn_predict = nn_model.predict(test_points)
print(mean_absolute_error(test_values, nn_predict))

Нейронные сети продемонcтрировали ухудшение качества модели при увеличении объема входных данных.

Попробуем собрать еще модель линейной регрессии.

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(train_points, train_values)
lr_predict = model.predict(test_points)
print(mean_absolute_error(test_values, lr_predict))

Метрика MAE говорит, что модель линейной регрессии справляется с задачей лучше, чем нейронные сети. Проверим этот результат графически.

In [ ]:
plt.figure(figsize=(7, 7))
plt.scatter(test_values, lr_predict) # рисуем точки, соответствущие парам настоящее значение - прогноз
plt.plot([0, max(test_values)], [0, max(lr_predict)], 'r') # рисуем прямую, на которой предсказания и настоящие значения совпадают
plt.xlabel('Настоящая цена', fontsize=20)
plt.ylabel('Предсказанная цена', fontsize=20);

Из-за излишней концентрации на вылетах модель линейной регрессии имеет тенденцию занижать цену для дорогих автомобилей. Связано это с тем, что наша выборка не является взвешенной, то есть имеет гораздо меньший объем записей об автомобилях с высокой ценой, чем с низкой.

## Построение регрессора для предсказания непрерывной величины (*Практическая* часть)

Вашим заданием в данной лабораторной будет построение регрессионной модели для предсказания цены квартиры для датасета из первой лабораторной работы. 

Доступные технологии:


*   XGBoost (n_estimators <= 100)
*   Random forest (n_estimators <= 100)
*   Нейронная сеть (epochs <= 15, общее количество слоев <= 5)
*   Линейная регрессия (Ну мало ли....)

Методика оценки - **mean_absolute_error**

In [ ]:
def plot_results(test_vals, predict_vals):
    plt.figure(figsize=(8, 6))
    plt.scatter(test_vals, predict_vals, linewidths=1, s=9)
    plt.plot([0, max(test_vals)], [0, max(predict_vals)], 'r')
    plt.xlabel('Истинные значения', fontsize=14)
    plt.ylabel('Предсказанные значения', fontsize=14);
    plt.grid(True)

In [ ]:
def print_results(estimator, X_train, y_train, X_test, y_test):
    print('MSE (train):', round(mean_squared_error(y_train, estimator.predict(X_train), squared=False), 6))
    print('MSE (test) :', round(mean_squared_error(y_test, estimator.predict(X_test), squared=False), 6))
    print('------------')
    print('MAE (train):', round(mean_absolute_error(y_train, estimator.predict(X_train)), 6))
    print('MAE (test) :', round(mean_absolute_error(y_test, estimator.predict(X_test)), 6))

In [ ]:
df = pd.read_csv('nyc-rolling-sales.csv')
df.info()

In [ ]:
df = pd.read_csv('nyc-rolling-sales.csv')
print('Default shape:', df.shape)

df = df.drop('Unnamed: 0', axis=1)
df = df.drop('ADDRESS', axis=1)
df = df.drop('APARTMENT NUMBER', axis=1)
df = df.drop('EASE-MENT', axis=1)
df = df.drop('SALE DATE', axis=1)

df = df.drop('ZIP CODE', axis=1)

# Удалим недвижимость с ценой продажи с '-' (нулевой ценой продажи)
df = df.drop(df[df['SALE PRICE'] == ' -  '].index)
df['SALE PRICE'] = df['SALE PRICE'].astype('int64')

# И с ценой продажей близкой к нулю (< 2000)
df = df.drop(df[df['SALE PRICE'] < 2000].index)

# И со слишком большой ценой продажи (> 3000000)
df = df.drop(df[df['SALE PRICE'] > 3000000].index)

# Заменим пропуски ' -  ' в LAND SQUARE FEET и GROSS SQUARE FEET
df['LAND SQUARE FEET'] = df['LAND SQUARE FEET'].replace(' -  ', -9999)
df['GROSS SQUARE FEET'] = df['GROSS SQUARE FEET'].replace(' -  ', -9999)
df['LAND SQUARE FEET'] = df['LAND SQUARE FEET'].astype('int64')
df['GROSS SQUARE FEET'] = df['GROSS SQUARE FEET'].astype('int64')


# Test
#df = df.drop('BOROUGH', axis=1)
#df = df.drop('BUILDING CLASS AT PRESENT', axis=1)

df = df.reset_index(drop=True)

In [ ]:
# Факторизация столбцов

cat_columns = ['NEIGHBORHOOD', 'BUILDING CLASS CATEGORY', 'TAX CLASS AT PRESENT',
               'BUILDING CLASS AT PRESENT', 'BUILDING CLASS AT TIME OF SALE']
num_columns = set(df.columns) - set(cat_columns)

for column in cat_columns:
    df[column] = pd.factorize(df[column])[0]

In [ ]:
df.sample(3)

In [ ]:
df.describe()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
cor_m = df.corr()
matr = np.triu(cor_m)
sns.heatmap(cor_m, mask=matr, linewidths=1, linecolor='white', vmin=-1, vmax=1, center= 0, cmap= 'coolwarm')

In [ ]:
sns.pairplot(df, diag_kind='hist', corner=True)

In [ ]:
y = df['SALE PRICE'].values
X = df.drop('SALE PRICE', axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [ ]:
rf100 = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=7)
rf100.fit(X_train, y_train)

In [ ]:
print_results(rf100, X_train, y_train, X_test, y_test)

In [ ]:
plot_results(y_test, rf100.predict(X_test))